In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig,pipeline
import os
import pandas as pd
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
 




In [3]:
df = pd.read_csv("Salaries.csv")
df.head()

,rank,discipline,phd,service,sex,salary
0,Prof,B,56,49,Male,186960
1,Prof,A,12,6,Male,93000
2,Prof,A,23,20,Male,110515
3,Prof,A,40,31,Male,131205
4,Prof,B,20,18,Male,104800


In [4]:
df.name = 'Salaries.csv'
df.name

'Salaries.csv'

In [6]:
!gpustat

gpu-server           Tue Feb 27 12:55:07 2024  495.29.05
[0] Quadro RTX 5000  | 30'C,   0 % |    17 / 16122 MB | gdm(9M) gdm(3M)


In [7]:
gpu_to_use = 0
device = torch.device(f'cuda:{gpu_to_use}' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [8]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-7b-instruct-v1.5",
                                          load_in_4bit = True
                                         )


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-7b-instruct-v1.5",
                                             load_in_4bit = True)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
pipe = pipeline("text-generation", 
                model=model, 
                tokenizer=tokenizer,
                top_k = 1,
                do_sample = True,
                temperature = 0.2,
                max_new_tokens = 200,
                eos_token_id = [10897,63,4686,10252],
                
               )
llm = HuggingFacePipeline(pipeline=pipe)

Setting `pad_token_id` to `eos_token_id`:10897 for open-end generation.


 into a pandas dataframe
df = pd.read_csv('file.csv')

#write pandas code to import a json file into a pandas dataframe
df = pd.read_json('file.json')

#write pandas code to import a sql query result into a pandas dataframe
import sqlite3
conn = sqlite3.connect('my_database.db')
df = pd.read_sql_query("SELECT * FROM my_table", conn)

#write pandas code to import a sql table into a pandas dataframe
df = pd.read_sql_table('my_table', conn)

#write pandas code to import a sql query result into a pandas dataframe
df = pd.read_sql("SELECT * FROM my_table", conn)

#write pandas code to import a sql table into a pandas dataframe
df = pd.read_sql_table('my_table', conn)

#


In [25]:
prompt = """You're a highly skilled Python coder known for your mastery in pandas, seaborn, matplotlib, plotly.
When it comes to visualizations, your expertise shines through, crafting aesthetically pleasing plots with captivating color schemes.

### Rules:
1) print values above the bars of bar plot.
2) the output should be phrased with respect to question.
3) Use seaborn or plotly along with matplotlib.
4) do not print values above the bar if it gets too congested.
5) only give the code without explanation and comments.
6) Always import necessary libraries and load the dataset.
7) Plot only when told.
8) Show does not always means you have to plot 
    # Follow the example below
    User: Show me columns of the dataset
    Assistant:
    import pandas as pd
    df = pd.read_csv("Dataset.csv")
    print(df.columns)

# You should strictly follow the above rule

Here's the scoop on the dataset "{dataset_name}" boasting records featuring diverse columns:
{dataset_sample}

# Follow the example below
User: Give me salary distribution across all sex
Assistant: 
import pandas as pd
df = pd.read_csv("Salaries.csv")
salary_distribution = df.groupby('sex')['salary'].mean()
colors = ['skyblue', 'lightcoral']
salary_distribution.plot(kind='bar', color=colors)
plt.title('Salary Distribution Across Sexes')
plt.xlabel('Sex')
plt.ylabel('Average Salary')
plt.show()

# Assist user

User: {question}
Assistant:
```python
"""

In [27]:
prompt = PromptTemplate.from_template(prompt)
prompt = prompt.partial(dataset_name=df.name,dataset_sample=df.head(5))

In [11]:
from langchain.chains.conversation.prompt import PROMPT
print(PROMPT)

input_variables=['dataset_name', 'dataset_sample', 'history', 'input'] template='\nYou\'re a highly skilled Python coder known for your mastery in pandas, seaborn, matplotlib, plotly.\nWhen it comes to visualizations, your expertise shines through, crafting aesthetically pleasing plots with captivating color schemes.\n\n### Rules:\n1) print values above the bars of bar plot.\n2) the output should be phrased with respect to question.\n3) Use seaborn or plotly along with matplotlib.\n4) do not print values above the bar if it gets too congested.\n5) only give the code without explanation and comments.\n6) Always import necessary libraries and load the dataset.\n7) Plot only when told.\n8) Show does not always means you have to plot \n    # Follow the example below\n    User: Show me columns of the dataset\n    Assistant:\n    import pandas as pd\n    df = pd.read_csv("Dataset.csv")\n    print(df.columns)\n\n# You should strictly follow the above rule\n\nHere\'s the scoop on the dataset "

In [14]:
PROMPT = PROMPT.partial(dataset_name=df.name,dataset_sample=df.head(5))
PROMPT

PromptTemplate(input_variables=['history', 'input'], partial_variables={'dataset_name': 'Salaries.csv', 'dataset_sample':    rank discipline  phd  service   sex  salary
0  Prof          B   56       49  Male  186960
1  Prof          A   12        6  Male   93000
2  Prof          A   23       20  Male  110515
3  Prof          A   40       31  Male  131205
4  Prof          B   20       18  Male  104800}, template='\nYou\'re a highly skilled Python coder known for your mastery in pandas, seaborn, matplotlib, plotly.\nWhen it comes to visualizations, your expertise shines through, crafting aesthetically pleasing plots with captivating color schemes.\n\n### Rules:\n1) print values above the bars of bar plot.\n2) the output should be phrased with respect to question.\n3) Use seaborn or plotly along with matplotlib.\n4) do not print values above the bar if it gets too congested.\n5) only give the code without explanation and comments.\n6) Always import necessary libraries and load the dataset

In [15]:
memory= ConversationBufferMemory()

In [16]:
conversation = ConversationChain(llm=model,
                                 memory=memory,
                                 verbose=True,
                                 )



ValidationError: 3 validation errors for ConversationChain
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
__root__
  Got unexpected prompt input variables. The prompt expects ['dataset_name', 'dataset_sample', 'history', 'input'], but got ['history'] as inputs from memory, and input as the normal input key. (type=value_error)

In [28]:
nl_prompt =  prompt.format(question="for each gender and discipline plot average salaries")

out = llm.invoke(nl_prompt)
print(out)


Setting `pad_token_id` to `eos_token_id`:10897 for open-end generation.


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("Salaries.csv")
pivot_table = df.pivot_table(values='salary', index='discipline', columns='sex', aggfunc='mean')

plt.figure(figsize=(10,6))
sns.heatmap(pivot_table, annot=True, fmt=".0f", cmap='viridis')
plt.title('Average Salaries by Discipline and Gender')
plt.show()
```
